In [233]:
from skimage.io import imread, imshow
import matplotlib.pyplot as plt
import numpy as np
from scipy.misc import imresize
import math
from scipy.ndimage.filters import convolve1d
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [218]:
img = imread('00_input/train/00000.png')

In [219]:
img_resized = imresize(img, size=(32, 32, 3))

In [220]:
def get_img_y(img):
    img_y = []
    for row in img:
        img_y.append(0.299 * row.T[0] + 0.587 * row.T[1] + 0.114 * row.T[2])
    return np.asarray(img_y)

def get_grad_x(img):
    return convolve1d(img, [1, 0, -1])

def get_grad(img):
    h, w = img.shape
    grad_x = get_grad_x(img)
    grad_y = get_grad_x(img.T).T
    grad_magn = np.sqrt(grad_x ** 2 + grad_y ** 2)
    grad_angl = np.arctan2(grad_y, grad_x)
    grad_uns_angl = grad_angl * (grad_angl >= 0) + (np.pi + grad_angl) * (grad_angl < 0)
    return grad_magn, grad_uns_angl

In [221]:
def get_hog(grad_magn, grad_angl, bins_number=9):
    hog = [0 for _ in range(bins_number)]
    bin_size = np.pi / bins_number
    for i in range(len(grad_angl)):
        for j in range(len(grad_angl[i])):
            grad_m = grad_magn[i][j]
            grad_a = grad_angl[i][j]
            bin_number = grad_a / bin_size
            bin_1_number = int(bin_number) % bins_number
            bin_2_number = (bin_1_number + 1) % bins_number
            hog[bin_1_number] += grad_m * abs(1 - bin_number + int(bin_number)) / bin_size
            hog[bin_2_number] += grad_m * abs(bin_number - int(bin_number)) / bin_size
    return hog

In [222]:
img_y = get_img_y(img_resized)
grad_magn, grad_angl = get_grad(img_y)

In [223]:
hog = get_hog(grad_magn, grad_angl)

In [224]:
def get_hog_for_cells(grad_magn, grad_angl, cell_size=8):
    n = len(grad_magn) 
    n_cell = 0
    if n % cell_size == 0:
        n_cell = int(n / cell_size)
    else:
        n_cell = int(n / cell_size) + 1
    hog_cells = [[] for _ in range(n_cell)]
    for i in range(0, n, cell_size):
        for j in range(0, n, cell_size):
            hog = get_hog(grad_magn[i:i + cell_size, j:j + cell_size],
                         grad_angl[i:i + cell_size, j:j + cell_size]) 
            hog_cells[int(i / cell_size)].append(hog)
    return np.asarray(hog_cells)

In [225]:
hog_cells = get_hog_for_cells(grad_magn, grad_angl)

In [226]:
def get_hog_for_blocks(hog_cells, block_size=3):
    n = len(hog_cells)
    eps = 10 ** (-9)
    hog_blocks = np.array([])
    for i in range(n - block_size + 1):
        for j in range(n - block_size + 1):
            normed_hog = np.array([])
            for ii in range(block_size):
                for jj in range(block_size):
                    normed_hog = np.concatenate([normed_hog, hog_cells[i][j]])
            normed_hog = normed_hog / (np.sqrt(np.sum(normed_hog ** 2)) + eps)
            hog_blocks = np.concatenate([hog_blocks, normed_hog])
    return hog_blocks    

In [227]:
hog_blocks = get_hog_for_blocks(hog_cells)

In [237]:
def get_features(img):
    img_resized = imresize(img, size=(32, 32, 3))
    img_y = get_img_y(img_resized)
    grad_magn, grad_angl = get_grad(img_y)
    hog_cells = get_hog_for_cells(grad_magn, grad_angl, cell_size=4)
    hog_blocks = get_hog_for_blocks(hog_cells, block_size=2)
    return hog_blocks

In [238]:
data = pd.read_csv('00_input/train/gt.csv')
print(data.head())

    filename  class_id
0  00000.png         0
1  00001.png         0
2  00002.png         0
3  00003.png         0
4  00004.png         0


In [239]:
%%time
X = []
y = []
for index, row in data.iterrows():
    filename = row['filename']
    class_id = int(row['class_id'])
    x = get_features(imread('00_input/train/' + filename))
    X.append(x)
    y.append(class_id)

CPU times: user 7min 41s, sys: 3.74 s, total: 7min 44s
Wall time: 13min 25s


In [247]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=42)

In [258]:
clf = LinearSVC(random_state=42, C=0.2, dual=False)
clf.fit(X_train, y_train)

LinearSVC(C=0.2, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

In [259]:
y_pred = clf.predict(X_test)

In [260]:
accuracy_score(y_test, y_pred)

0.95610170801452976

In [261]:
accuracy_score(y_train, clf.predict(X_train))

0.98374571754853446